In [56]:
import pandas as pd
import numpy as np
# library to clean data
import re
# Natural Language Tool Kit
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
# to remove stopword
from nltk.corpus import stopwords
# for Stemming propose
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from joblib import dump, load

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\I353174\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\I353174\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [257]:
df = pd.read_csv('salesleads.csv') 
df.head()

,email,name,title,label,content
0,Info@1015multimedia.com,Rishikumar Thakur,NaN,warm,"Hello, I have had software in my warehouse for 2 days under Export-File Options an almost illegible font. How can I do this again to change."
1,info@142productions.com,Raza ji,NaN,warm,"I have all sales firstclass versions from 2006 until today. On Saturday got the version WAREHOUSE Sales firstclass 20 Neo, had to be installed after the installation notice that the label of the buttons in the export screen are not readable (e.g. file format, navigation, compression, font, sort by etc. design the complete labeling under file options, title page and Design buttons. After an uninstallation and a new Installation has not shown in the export screen changed. Since I conduct warehouse training for company directors, a Proper work not possible."
2,info@223agency.com,Vaibhav Patel,NaN,hot,"Hello, I have 3 questions about ""Sales firstclass WH Neo"": 1. In the overview mode, I marked part of the process with the process marker (blue line with start and end markings). The export routine exports the entire process, including what is outside of the process marker. Is that a program error (in ""Sales firstclass 2008"" only the process part between the start and end pointer is correctly burned)? 2. In export mode, the font in the small preview window in the file menu is always identical to the font in the heading line from the submenu. How can I break this link, or is it a program error? 3. Are there complete instructions, possibly to download or order? Kind regards. Miraculix"
3,scott@32digital.com,Amazon Customer,NaN,warm,"On my specification sheet, the text modules are only displayed as a speech bubble, I should have had an online presentation of my specificationsheets today to win a sponsor. This has now jumped off! Thanks for such a great technology, I'm pissed off! I hope and expect that the specification sheet will be operational again immediately, which is a shame so far I could rely on Warehouse halfway! I ask for an answer that won't come back Takes 21 days!"
4,cservice@4hatteras.com,Sudhakaran Wadakkancheri,NaN,cold,"Since your update, strange bubbles have appeared on my sheet, but I haven't have cleaned!"


In [258]:
# To Count number of null values in column
def count_null(*args):
    for col in args:
        column=df[col].isnull().sum(axis=0)
    return column

In [259]:
count_null(list(df.columns[:]))

email      260
name       347
title      627
label        0
content      0
dtype: int64

In [260]:
pd.set_option('display.max_colwidth', None) 
df[['content']].head(5)

,content
0,"Hello, I have had software in my warehouse for 2 days under Export-File Options an almost illegible font. How can I do this again to change."
1,"I have all sales firstclass versions from 2006 until today. On Saturday got the version WAREHOUSE Sales firstclass 20 Neo, had to be installed after the installation notice that the label of the buttons in the export screen are not readable (e.g. file format, navigation, compression, font, sort by etc. design the complete labeling under file options, title page and Design buttons. After an uninstallation and a new Installation has not shown in the export screen changed. Since I conduct warehouse training for company directors, a Proper work not possible."
2,"Hello, I have 3 questions about ""Sales firstclass WH Neo"": 1. In the overview mode, I marked part of the process with the process marker (blue line with start and end markings). The export routine exports the entire process, including what is outside of the process marker. Is that a program error (in ""Sales firstclass 2008"" only the process part between the start and end pointer is correctly burned)? 2. In export mode, the font in the small preview window in the file menu is always identical to the font in the heading line from the submenu. How can I break this link, or is it a program error? 3. Are there complete instructions, possibly to download or order? Kind regards. Miraculix"
3,"On my specification sheet, the text modules are only displayed as a speech bubble, I should have had an online presentation of my specificationsheets today to win a sponsor. This has now jumped off! Thanks for such a great technology, I'm pissed off! I hope and expect that the specification sheet will be operational again immediately, which is a shame so far I could rely on Warehouse halfway! I ask for an answer that won't come back Takes 21 days!"
4,"Since your update, strange bubbles have appeared on my sheet, but I haven't have cleaned!"


In [261]:
def process_docs(docs, rows) :
    corpus = []
    for i in range(0, rows):
        # column : "content", row ith
        content = re.sub('[^a-zA-Z]', ' ', docs[i])
    
        # convert all cases to lower cases
        content = content.lower()
     
        # split to array(default delimiter is " ")
        content = content.split()
    
        # ps = PorterStemmer()
        lem = WordNetLemmatizer()
    
        content = [lem.lemmatize(word) for word in content
                if not word in set(stopwords.words('english'))]
        
        content = ' '.join(content) 
        corpus.append(content)
        
    return corpus  

In [262]:
corpus = process_docs(df['content'], df.shape[0])
df['clean_text'] = corpus
le = LabelEncoder()
df.label = le.fit_transform(df.label)
df.head()

,email,name,title,label,content,clean_text
0,Info@1015multimedia.com,Rishikumar Thakur,NaN,2,"Hello, I have had software in my warehouse for 2 days under Export-File Options an almost illegible font. How can I do this again to change.",hello software warehouse day export file option almost illegible font change
1,info@142productions.com,Raza ji,NaN,2,"I have all sales firstclass versions from 2006 until today. On Saturday got the version WAREHOUSE Sales firstclass 20 Neo, had to be installed after the installation notice that the label of the buttons in the export screen are not readable (e.g. file format, navigation, compression, font, sort by etc. design the complete labeling under file options, title page and Design buttons. After an uninstallation and a new Installation has not shown in the export screen changed. Since I conduct warehouse training for company directors, a Proper work not possible.",sale firstclass version today saturday got version warehouse sale firstclass neo installed installation notice label button export screen readable e g file format navigation compression font sort etc design complete labeling file option title page design button uninstallation new installation shown export screen changed since conduct warehouse training company director proper work possible
2,info@223agency.com,Vaibhav Patel,NaN,1,"Hello, I have 3 questions about ""Sales firstclass WH Neo"": 1. In the overview mode, I marked part of the process with the process marker (blue line with start and end markings). The export routine exports the entire process, including what is outside of the process marker. Is that a program error (in ""Sales firstclass 2008"" only the process part between the start and end pointer is correctly burned)? 2. In export mode, the font in the small preview window in the file menu is always identical to the font in the heading line from the submenu. How can I break this link, or is it a program error? 3. Are there complete instructions, possibly to download or order? Kind regards. Miraculix",hello question sale firstclass wh neo overview mode marked part process process marker blue line start end marking export routine export entire process including outside process marker program error sale firstclass process part start end pointer correctly burned export mode font small preview window file menu always identical font heading line submenu break link program error complete instruction possibly download order kind regard miraculix
3,scott@32digital.com,Amazon Customer,NaN,2,"On my specification sheet, the text modules are only displayed as a speech bubble, I should have had an online presentation of my specificationsheets today to win a sponsor. This has now jumped off! Thanks for such a great technology, I'm pissed off! I hope and expect that the specification sheet will be operational again immediately, which is a shame so far I could rely on Warehouse halfway! I ask for an answer that won't come back Takes 21 days!",specification sheet text module displayed speech bubble online presentation specificationsheets today win sponsor jumped thanks great technology pissed hope expect specification sheet operational immediately shame far could rely warehouse halfway ask answer come back take day
4,cservice@4hatteras.com,Sudhakaran Wadakkancheri,NaN,0,"Since your update, strange bubbles have appeared on my sheet, but I haven't have cleaned!",since update strange bubble appeared sheet cleaned


In [263]:
y = df["label"]
y.value_counts()

0    243
1    234
2    150
Name: label, dtype: int64

In [278]:
features = 3000
vectorizer = None

def transform_text(transformer, corpus): 
    global vectorizer
    if transformer == "countVectorizer":
        vectorizer = CountVectorizer(max_features=features)
        vectorizer.fit(corpus)  
        vector = vectorizer.transform(corpus)
        X = vector.toarray()
        
    if transformer == "Tfidf":
        vectorizer = TfidfVectorizer(max_features=features)
        vectorizer.fit(corpus)  
        vector = vectorizer.transform(corpus)
        X = vector.toarray()
        
    dump(vectorizer, "vectorizer.pkl")
    return X    

In [279]:
X = transform_text("Tfidf", corpus)
print('shape of X:', X.shape)
print('shape of y:', y.shape)

shape of X: (627, 3000)
shape of y: (627,)


In [288]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


def train_model(model_name):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    print("Shape of X_train:", X_train.shape)
    print("Shape of X_test:", X_test.shape)
    if model_name == "MultinomialNB":
        ml_model = MultinomialNB()
        ml_model.fit(X_train, y_train)
        y_pred = ml_model.predict(X_test)
        print("Accuracy of MultinomialNB:", metrics.accuracy_score(y_test, y_pred))  
        
    if model_name == "GaussianNB":
        ml_model = GaussianNB()
        ml_model.fit(X_train, y_train)
        y_pred = ml_model.predict(X_test)
        print("Accuracy of GaussianNB:", metrics.accuracy_score(y_test, y_pred))
        
    if model_name == "SVC":
        ml_model = make_pipeline(StandardScaler(), SVC(gamma='auto'))
        ml_model.fit(X_train, y_train)
        y_pred = ml_model.predict(X_test)
        print("Accuracy of SVC:", metrics.accuracy_score(y_test, y_pred)) 
        
    if model_name == "RandomForestClassifier":
        params = {'bootstrap': True, 'max_depth': 90, 'max_features': 3, 'min_samples_leaf': 4, 'min_samples_split': 12, 'n_estimators': 100}
        ml_model = RandomForestClassifier(**params)
        ml_model.fit(X_train, y_train)
        y_pred = ml_model.predict(X_test)
        print("Accuracy of RandomForestClassifier:", metrics.accuracy_score(y_test, y_pred))     
        
    dump(ml_model, "model.pkl")    
    return ml_model 

In [294]:
ml_model = train_model("RandomForestClassifier")

Shape of X_train: (438, 3000)
Shape of X_test: (189, 3000)
Accuracy of RandomForestClassifier: 0.36507936507936506


In [290]:
vectorizer = load('vectorizer.pkl') 
model = load('model.pkl') 